In [1]:
from hybrid_recommenders.experiment.combined_user_cf import combined_user_cf
from utils.evaluator.evaluator import evaluator
from utils.submission_helper import submission_helper
from utils.data_manager.data_manager import data_manager
from utils.data_manager.splitter import splitter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
training, validation, test = splitter().get_train_evaluation_test(percentage_of_training_data=0.6,
                                                                  percentage_of_validation_data=0.2)



In [3]:
k_vector = [50,100,150,200,250,300,350,400,450,500,550]
similarities = ['tanimoto', 'dice', 'cosine', 'jaccard']
shrink_vector = [1,3,5,7,9,11,13,15,18]
similarities_array = []
parameters_vector = []
map_vector = []
previous_similarity = None
for i in range(0,4):
    similarities_array.append([])
for similarity in similarities:
    for shrink in shrink_vector:
        for k in k_vector:
            parameters_vector.append([k, similarity, shrink])
recommender = combined_user_cf()
recommender.fit(training_set=training,
                k=parameters_vector[0][0],
                similarity=parameters_vector[0][1],
                shrink=parameters_vector[0][2],
                rating_per_user=3)
map_ = evaluator.evaluate_already_trained(recommender=recommender)[2]
map_vector.append([map_, parameters_vector[0]])
print("map: " ,map_, " parameters: ", parameters_vector[0], " = [k, similarity, shrink]")
for params in parameters_vector[1:]:
    recommender.change_model_parameters(new_k=params[0],
                                        new_similarity=params[1],
                                        new_shrink=params[2])
    map_ = evaluator.evaluate_already_trained(recommender=recommender)[2]
    map_vector.append([map_, params])
    print("map: " ,map_, " parameters: ", params, " = [k, similarity, shrink]")
map_vector




Similarity column 25975 ( 100 % ), 1536.54 column/sec, elapsed time 0.28 min
ratings per user:  3
the new URM matrix is ready. Shape:  (7947, 25975)
Similarity column 7947 ( 100 % ), 2671.36 column/sec, elapsed time 0.05 min
map:  0.0289434165631691  parameters:  [50, 'tanimoto', 1]  = [k, similarity, shrink]
Similarity column 7947 ( 100 % ), 3202.94 column/sec, elapsed time 0.04 min
map:  0.030690657604974865  parameters:  [100, 'tanimoto', 1]  = [k, similarity, shrink]
Similarity column 7947 ( 100 % ), 3212.63 column/sec, elapsed time 0.04 min
map:  0.03155177112319196  parameters:  [150, 'tanimoto', 1]  = [k, similarity, shrink]
Similarity column 7947 ( 100 % ), 3075.05 column/sec, elapsed time 0.04 min
map:  0.03196950909323412  parameters:  [200, 'tanimoto', 1]  = [k, similarity, shrink]
Similarity column 7947 ( 100 % ), 2975.38 column/sec, elapsed time 0.04 min
map:  0.03218966432273724  parameters:  [250, 'tanimoto', 1]  = [k, similarity, shrink]
Similarity column 7947 ( 100 % )

[[0.0289434165631691, [50, 'tanimoto', 1]],
 [0.030690657604974865, [100, 'tanimoto', 1]],
 [0.03155177112319196, [150, 'tanimoto', 1]],
 [0.03196950909323412, [200, 'tanimoto', 1]],
 [0.03218966432273724, [250, 'tanimoto', 1]],
 [0.032172270209821434, [300, 'tanimoto', 1]],
 [0.03217230246639562, [350, 'tanimoto', 1]],
 [0.03211082612999389, [400, 'tanimoto', 1]],
 [0.03197932954759398, [450, 'tanimoto', 1]],
 [0.0319621625374356, [500, 'tanimoto', 1]],
 [0.03197668125076976, [550, 'tanimoto', 1]],
 [0.02888571407857909, [50, 'tanimoto', 3]],
 [0.03106628716039703, [100, 'tanimoto', 3]],
 [0.03157299883180637, [150, 'tanimoto', 3]],
 [0.032067966533705895, [200, 'tanimoto', 3]],
 [0.03228967199107097, [250, 'tanimoto', 3]],
 [0.03224596575182744, [300, 'tanimoto', 3]],
 [0.032257141641039794, [350, 'tanimoto', 3]],
 [0.03219161688362495, [400, 'tanimoto', 3]],
 [0.03210524329818526, [450, 'tanimoto', 3]],
 [0.03207765907777939, [500, 'tanimoto', 3]],
 [0.03210211381399506, [550, 'tani

In [27]:
similarities = ['tanimoto', 'dice', 'cosine', 'jaccard']
map_vectors = [[array[0] for array in map_vector if array[1][1]== similarity] for similarity in similarities]
parameters_vector = [[array[1] for array in map_vector if array[1][1]==similarity]for similarity in similarities]
max_map_values = [np.max(array) for array in map_vectors]
best_parameters_vector = []
for i in range(0,len(parameters_vector)):
    params = parameters_vector[i]
    map_values = map_vectors[i]
    to_append = params[np.argmax(map_values)]
    best_parameters_vector.append(to_append)
for params, map_ in list(zip(best_parameters_vector, max_map_values)):
    print("map: ", map_,"  with parameters: ", params)


map:  0.03312423181181323   with parameters:  [250, 'tanimoto', 18]
map:  0.03318974200425683   with parameters:  [250, 'dice', 15]
map:  0.029856545412275195   with parameters:  [250, 'cosine', 3]
map:  0.03312423181181323   with parameters:  [250, 'jaccard', 18]


In [4]:
weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
map_vector = []
recommender = combined_user_cf(weight=weights[0])
recommender.fit(training_set=training.copy(),
                k_1=250,
                shrink_1=18,
                similarity_1='tanimoto',
                k_2=250,
                shrink_2=18,
                similarity_2='jaccard',
                k_total=None,
                rating_per_user=3)
map_ = evaluator.evaluate_already_trained(recommender=recommender)[2]
map_vector.append(map_)
print(map_ , " ", weights[0])
for w in weights[1:]:
    recommender.change_weight(new_weight=w)
    map_ = evaluator.evaluate_already_trained(recommender=recommender)[2]
    map_vector.append(map_)
    print(map_ , " ", w)
best_weight = weights[np.argmax(map_vector)]
max_map = np.max(map_vector)
print("\n\nbest map score: ", max_map, " with this weight: ", best_weight)

Similarity column 25975 ( 100 % ), 1198.19 column/sec, elapsed time 0.36 min
ratings per user:  3
the new URM matrix is ready. Shape:  (7947, 25975)
Similarity column 7947 ( 100 % ), 3208.31 column/sec, elapsed time 0.04 min
Similarity column 7947 ( 100 % ), 3278.46 column/sec, elapsed time 0.04 min
0.033123782406017696   0.1
0.033123782406017696   0.2
0.033123782406017696   0.3
0.033123782406017696   0.4
0.033123782406017696   0.5
0.033123782406017696   0.6
0.033123782406017696   0.7
0.033123782406017696   0.8
0.033123782406017696   0.9


best map score:  0.033123782406017696  with this weight:  0.1
